## Initizialization

In [1]:
#!pip install keras
#!pip install tensorflow
#!pip install -U imbalanced-learn, scikit-learn
#!pip install xgboost

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data preparation

In [38]:
# load data and set IVH level to binary 0 or 1
df = pd.read_csv('Table_preterm_2019_articleMLM.csv', sep=';')
df.columns = df.columns.str.lower().str.replace(' ','_') 
df.columns = df.columns.str.lower().str.replace(':','_') 
df.columns = df.columns.str.lower().str.replace(',','') 
df.columns = df.columns.str.lower().str.replace('-','_') 
df.columns = df.columns.str.lower().str.replace('/','_or_') 
df.columns = df.columns.str.lower().str.replace('#','') 

## Exploratory Analisis

In [118]:
#df

In [119]:
#df.T

In [ ]:
df.head(5)

In [ ]:
#df.gender.value_counts()
#df.birth__1_natural_2_caesarean.value_counts()

In [ ]:
for col in df.columns:
    print(col)
    print(df[col].unique()[:5])
    print(df[col].nunique())
    print()

In [57]:
degree_ivh = df.groupby('patient').grad_ivh
# degree of the ivh
vc = degree_ivh.value_counts()
vc

patient  grad_ivh
1        3           72
2        3           31
3        2           34
4        3           12
5        3           34
                     ..
312      0           12
313      0            5
314      0            7
315      0            8
316      0           17
Name: count, Length: 316, dtype: int64

In [65]:
df[df.grad_ivh>0].iloc[:,0]

0         1
1         1
2         1
3         1
4         1
       ... 
3269    146
3270    146
3271    146
3272    146
3273    146
Name: patient, Length: 3274, dtype: int64

In [53]:
degree_ivh = df.groupby('patient').grad_ivh.max()
# degree of the ivh
vc = degree_ivh.value_counts()
vc
#.churn.mean())
#print('global', global_churn_rate)

grad_ivh
0    170
3     49
2     45
1     44
4      8
Name: count, dtype: int64

### Data preprocessing

In [133]:
df_n = df[df.day_of_ivh == 0]
df_p = df[df.day_of_ivh > 1] # bigger than 1 cause not enough data
df_p_f = df_p[df_p.day_befor_or_after_ivh < 0]
dff = pd.concat([df_p_f,df_n], ignore_index=True)

In [134]:
dff['grad_ivh'] = dff['grad_ivh'].replace([1,2,3,4],1)

In [135]:
## dff.groupby('patient').mean()


In [136]:
#dff.grad_ivh.value_counts(normalize=True) # rate

In [137]:
#del dff['grad_ivh']
del dff['day_of_ivh']
del dff['day_befor_or_after_ivh']

In [160]:
list(dff.columns)

['patient',
 'wg',
 'gender',
 'body_weight',
 'grad_ivh',
 'birth__1_natural_2_caesarean',
 'number_of_children',
 'apgar_1min',
 'apgar_5min',
 'apgar_10min',
 'day_pf_life',
 'ph',
 'pco2',
 'po2',
 'sao2',
 'map',
 'leukocytes',
 'hematocrit',
 'thrombocytes',
 'crp']

In [ ]:
no_agg_col = [
    'wg', 'gender', 'body_weight', 'grad_ivh', 
    'birth__1_natural_2_caesarean',
    'number_of_children'
    ]

In [138]:

dff_aggregated = dff.groupby('patient').agg(['mean', 'std', 'min', 'max']).reset_index()
# Flattening the multi-level column names
dff_aggregated.columns = ['_'.join(col) for col in dff_aggregated.columns]

In [139]:
del dff_aggregated['patient_']
del dff_aggregated['wg_std']
del dff_aggregated['wg_min']
del dff_aggregated['wg_max']

del dff_aggregated['grad_ivh_min']
del dff_aggregated['grad_ivh_min']
del dff_aggregated['grad_ivh_min']

del dff_aggregated['grad_ivh_max']
del dff_aggregated['grad_ivh_max']
del dff_aggregated['grad_ivh_max']
dff_aggregated

,wg_mean,wg_std,wg_min,wg_max,gender_mean,gender_std,gender_min,gender_max,body_weight_mean,body_weight_std,...,hematocrit_min,hematocrit_max,thrombocytes_mean,thrombocytes_std,thrombocytes_min,thrombocytes_max,crp_mean,crp_std,crp_min,crp_max
0,23.5,0.0,23.5,23.5,2.0,0.0,2,2,490.0,0.0,...,0.000,0.508,169.444444,19.184050,134.0,197.0,0.077778,0.044096,0.0,0.1
1,24.4,0.0,24.4,24.4,2.0,0.0,2,2,335.0,0.0,...,0.000,0.327,360.000000,311.769145,0.0,540.0,0.300000,0.173205,0.2,0.5
2,24.4,0.0,24.4,24.4,1.0,0.0,1,1,850.0,0.0,...,0.507,0.654,236.666667,40.648083,197.0,316.0,0.383333,0.312517,0.1,0.7
3,24.1,0.0,24.1,24.1,2.0,0.0,2,2,710.0,0.0,...,0.405,0.443,216.818182,34.778781,174.0,247.0,0.390909,0.403620,0.1,0.9
4,24.4,0.0,24.4,24.4,2.0,0.0,2,2,650.0,0.0,...,0.000,0.517,214.571429,96.155875,0.0,269.0,0.100000,0.000000,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,33.3,0.0,33.3,33.3,1.0,0.0,1,1,1860.0,0.0,...,0.000,0.525,74.750000,110.579568,0.0,234.0,0.375000,0.226134,0.0,0.5
294,33.4,0.0,33.4,33.4,2.0,0.0,2,2,1960.0,0.0,...,0.000,0.384,148.800000,203.852152,0.0,381.0,0.200000,0.273861,0.0,0.5
295,33.1,0.0,33.1,33.1,1.0,0.0,1,1,1860.0,0.0,...,0.000,0.526,71.142857,121.499559,0.0,249.0,0.357143,0.243975,0.0,0.5
296,33.4,0.0,33.4,33.4,1.0,0.0,1,1,1900.0,0.0,...,0.000,0.489,81.125000,95.330159,0.0,214.0,0.312500,0.258775,0.0,0.5


In [132]:
categorical_columns = list(dff_aggregated.dtypes[dff_aggregated.dtypes == 'object'].index)
categorical_columns

[]

## Val split

In [140]:
from sklearn.model_selection import train_test_split

In [141]:
#train_test_split?#
df_full_train , df_test = train_test_split(dff_aggregated, test_size=0.2, random_state=1)
df_train , df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [142]:
len(df_train), len(df_val), len(df_test)

(178, 60, 60)

In [143]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [145]:
df_train.columns

Index(['wg_mean', 'wg_std', 'wg_min', 'wg_max', 'gender_mean', 'gender_std',
       'gender_min', 'gender_max', 'body_weight_mean', 'body_weight_std',
       'body_weight_min', 'body_weight_max', 'grad_ivh_mean', 'grad_ivh_std',
       'grad_ivh_min', 'grad_ivh_max', 'birth__1_natural_2_caesarean_mean',
       'birth__1_natural_2_caesarean_std', 'birth__1_natural_2_caesarean_min',
       'birth__1_natural_2_caesarean_max', 'number_of_children_mean',
       'number_of_children_std', 'number_of_children_min',
       'number_of_children_max', 'apgar_1min_mean', 'apgar_1min_std',
       'apgar_1min_min', 'apgar_1min_max', 'apgar_5min_mean', 'apgar_5min_std',
       'apgar_5min_min', 'apgar_5min_max', 'apgar_10min_mean',
       'apgar_10min_std', 'apgar_10min_min', 'apgar_10min_max',
       'day_pf_life_mean', 'day_pf_life_std', 'day_pf_life_min',
       'day_pf_life_max', 'ph_mean', 'ph_std', 'ph_min', 'ph_max', 'pco2_mean',
       'pco2_std', 'pco2_min', 'pco2_max', 'po2_mean', 'po2_std',

In [146]:
y_train = df_train.grad_ivh_mean.values
y_val = df_val.grad_ivh_mean.values
y_test = df_test.grad_ivh_mean.values
# grad_ivh_mean', 'grad_ivh_std', 'grad_ivh_min', 'grad_ivh_max'

In [ ]:
del df_train['grad_ivh_mean']
del df_val['grad_ivh_mean']
del df_test['grad_ivh_mean']

In [148]:
df_full_train.grad_ivh_mean.value_counts(normalize=True) # churn rate

grad_ivh_mean
0.0    0.554622
1.0    0.445378
Name: proportion, dtype: float64

Now that the dataset is aggregated, it is balanced

In [152]:
global_rate = df_full_train.grad_ivh_mean.mean()
global_rate

0.44537815126050423

In [151]:
from IPython.display import display

In [157]:
features = list(df_full_train.columns)
features

['wg_mean',
 'wg_std',
 'wg_min',
 'wg_max',
 'gender_mean',
 'gender_std',
 'gender_min',
 'gender_max',
 'body_weight_mean',
 'body_weight_std',
 'body_weight_min',
 'body_weight_max',
 'grad_ivh_mean',
 'grad_ivh_std',
 'grad_ivh_min',
 'grad_ivh_max',
 'birth__1_natural_2_caesarean_mean',
 'birth__1_natural_2_caesarean_std',
 'birth__1_natural_2_caesarean_min',
 'birth__1_natural_2_caesarean_max',
 'number_of_children_mean',
 'number_of_children_std',
 'number_of_children_min',
 'number_of_children_max',
 'apgar_1min_mean',
 'apgar_1min_std',
 'apgar_1min_min',
 'apgar_1min_max',
 'apgar_5min_mean',
 'apgar_5min_std',
 'apgar_5min_min',
 'apgar_5min_max',
 'apgar_10min_mean',
 'apgar_10min_std',
 'apgar_10min_min',
 'apgar_10min_max',
 'day_pf_life_mean',
 'day_pf_life_std',
 'day_pf_life_min',
 'day_pf_life_max',
 'ph_mean',
 'ph_std',
 'ph_min',
 'ph_max',
 'pco2_mean',
 'pco2_std',
 'pco2_min',
 'pco2_max',
 'po2_mean',
 'po2_std',
 'po2_min',
 'po2_max',
 'sao2_mean',
 'sao2_st

In [159]:
for c in features:
    df_group = df_full_train.groupby(c).grad_ivh_mean.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_rate
    df_group['risk'] = df_group['mean'] / global_rate
    display(df_group)

,mean,count,diff,risk
wg_mean,,,,
22.4,1.000000,1,0.554622,2.245283
23.0,0.000000,1,-0.445378,0.000000
23.1,0.333333,3,-0.112045,0.748428
23.2,0.666667,3,0.221289,1.496855
23.3,0.000000,1,-0.445378,0.000000
...,...,...,...,...
32.6,0.000000,3,-0.445378,0.000000
33.0,0.000000,2,-0.445378,0.000000
33.3,0.000000,2,-0.445378,0.000000


,mean,count,diff,risk
wg_std,,,,
0.000000,0.4329,231,-0.012478,0.971984
0.047140,0.0000,1,-0.445378,0.000000
0.122474,1.0000,1,0.554622,2.245283
0.122474,1.0000,1,0.554622,2.245283


,mean,count,diff,risk
wg_min,,,,
22.4,1.000000,1,0.554622,2.245283
23.0,0.000000,1,-0.445378,0.000000
23.1,0.333333,3,-0.112045,0.748428
23.2,0.666667,3,0.221289,1.496855
23.3,0.500000,2,0.054622,1.122642
...,...,...,...,...
32.6,0.000000,3,-0.445378,0.000000
33.0,0.000000,2,-0.445378,0.000000
33.3,0.000000,2,-0.445378,0.000000


,mean,count,diff,risk
wg_max,,,,
22.4,1.000000,1,0.554622,2.245283
23.0,0.000000,1,-0.445378,0.000000
23.1,0.333333,3,-0.112045,0.748428
23.2,0.666667,3,0.221289,1.496855
23.3,0.000000,1,-0.445378,0.000000
...,...,...,...,...
32.6,0.000000,3,-0.445378,0.000000
33.0,0.000000,2,-0.445378,0.000000
33.3,0.000000,2,-0.445378,0.000000


,mean,count,diff,risk
gender_mean,,,,
0.0,1.000000,1,0.554622,2.245283
1.0,0.472868,129,0.027490,1.061723
2.0,0.407407,108,-0.037971,0.914745


...

,mean,count,diff,risk
thrombocytes_max,,,,
0.0,0.818182,11,0.372804,1.837050
71.0,1.000000,1,0.554622,2.245283
95.0,1.000000,1,0.554622,2.245283
96.0,1.000000,1,0.554622,2.245283
99.0,1.000000,1,0.554622,2.245283
...,...,...,...,...
512.0,0.000000,1,-0.445378,0.000000
527.0,0.000000,1,-0.445378,0.000000
536.0,0.000000,1,-0.445378,0.000000


,mean,count,diff,risk
crp_mean,,,,
0.000000,0.852941,34,0.407563,1.915094
0.012500,1.000000,1,0.554622,2.245283
0.021739,0.000000,1,-0.445378,0.000000
0.022222,1.000000,1,0.554622,2.245283
0.035714,1.000000,1,0.554622,2.245283
...,...,...,...,...
1.627273,1.000000,1,0.554622,2.245283
2.818750,1.000000,1,0.554622,2.245283
3.200000,1.000000,1,0.554622,2.245283


,mean,count,diff,risk
crp_std,,,,
0.000000,0.734694,49,0.289316,1.649596
0.032280,0.000000,1,-0.445378,0.000000
0.035355,1.000000,1,0.554622,2.245283
0.040825,0.000000,1,-0.445378,0.000000
0.042174,0.000000,1,-0.445378,0.000000
...,...,...,...,...
2.633333,1.000000,1,0.554622,2.245283
3.843691,1.000000,1,0.554622,2.245283
4.359544,1.000000,1,0.554622,2.245283


,mean,count,diff,risk
crp_min,,,,
0.0,0.418719,203,-0.026659,0.940143
0.1,0.592593,27,0.147214,1.330538
0.2,1.000000,1,0.554622,2.245283
0.5,0.571429,7,0.126050,1.283019


,mean,count,diff,risk
crp_max,,,,
0.0,0.852941,34,0.407563,1.915094
0.1,0.611111,18,0.165733,1.372117
0.2,0.375000,8,-0.070378,0.841981
0.3,0.000000,3,-0.445378,0.000000
0.4,0.750000,4,0.304622,1.683962
0.5,0.277778,90,-0.167600,0.623690
0.6,0.333333,6,-0.112045,0.748428
0.7,0.400000,5,-0.045378,0.898113
0.8,0.000000,1,-0.445378,0.000000


In [ ]:
# create a binary label


In [36]:
neg_count = vc[0]
pos_count = vc[1:4].sum()
total = 
neg_count, pos_count, 


(3263, 3123)

## Next

In [ ]:
df['grad IVH'] = df['grad IVH'].replace([1,2,3,4],1)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures

In [5]:
import imblearn
from imblearn.over_sampling import SMOTE

In [6]:
data.head()

,#Patient,WG,Gender,Body weight,grad IVH,day of IVH,"Birth: 1-Natural, 2-Caesarean",Number of children,Apgar 1min,Apgar 5min,...,Day befor/after IVH,pH,pCO2,pO2,SaO2,MAP,Leukocytes,Hematocrit,Thrombocytes,CRP
0,1,23.5,2,490,1,3,2,1,1.0,1,...,-2.2,0.00,0.0,0.0,0.0,0,8.06,0.000,197.0,0.0
1,1,23.5,2,490,1,3,2,1,1.0,1,...,-2.1,0.00,0.0,0.0,0.0,0,8.06,0.464,197.0,0.0
2,1,23.5,2,490,1,3,2,1,1.0,1,...,-1.7,7.30,36.1,47.2,0.0,0,9.83,0.444,170.0,0.1
3,1,23.5,2,490,1,3,2,1,1.0,1,...,-1.6,7.23,49.1,50.9,91.0,22,9.83,0.444,170.0,0.1
4,1,23.5,2,490,1,3,2,1,1.0,1,...,-1.5,7.14,52.7,49.7,30.0,23,9.83,0.444,170.0,0.1


The goal is to be able to predict which child is going to get a IVH based on values prior to a IVH.
However cutting the data to the day of IVH reduces the number of data, and maybe they do contain valuable info. 

Approach:
Try to remove all datapoints after a IVH

In [7]:
X = data.drop(['grad IVH', 'day of IVH', 'Day befor/after IVH', '#Patient'], axis=1)  # Replace 'target_column_name' with the actual column name of the labels
y = data['grad IVH']

'''X = data.iloc[:, :-1].values
X = data.take([5], axis=1)
y = data.iloc[:, 5].values'''

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
poly_features = PolynomialFeatures(degree=2)
X_poly = poly_features.fit_transform(X)

# Convert the scaled features back to a DataFrame
X = pd.DataFrame( X_scaled, columns=X.columns)


Shape of X: (6537, 18)
Shape of y: (6537,)


In [8]:
#pip install ydata-synthetic

In [9]:

import sys
print(sys.version)

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [10]:
#num_classes = 2
unique_patients = np.unique(data.iloc[:, 0].values)
X_list = [X[data.iloc[:, 0].values == patient] for patient in unique_patients]
y_list = [np.array(y[data.iloc[:, 0].values == patient]) for patient in unique_patients]
#y_list = np.array(y_list)

X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=0.1, random_state=42)

In [11]:
X_train_ls = [X_train[i] for i in range(len(X_train))]
X_train= pd.concat(X_train_ls, ignore_index=True)
X_test_ls = [X_test[i] for i in range(len(X_test))]
X_test= pd.concat(X_test_ls, ignore_index=True)

#X_train= pd.concat(X_train_ls, ignore_index=True)

In [12]:
y_train_ls = [y_train[i].tolist() for i in range(len(y_train))]
y_train = sum(y_train_ls, [])
y_test_ls = [y_test[i].tolist() for i in range(len(y_test))]
y_test = sum(y_test_ls, [])
#type(y_train_ls)

In [13]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Model training

In [14]:
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
# Define the AdaBoost classifier
adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)

# Define the hyperparameters to tune
param_grid = {
    #'n_estimators': [7, 10, 50],
    #'learning_rate': [0.01, 0.05, 0.1],
    #'estimator__max_depth': [1, 2, 3]
    #estimator__min_samples_split": [2, 4, 6]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(adaboost, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

TypeError: AdaBoostClassifier.__init__() got an unexpected keyword argument 'base_estimator'

In [21]:
'''# Bagging
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)
bagging_predictions = bagging_model.predict(X_test)
bagging_accuracy = accuracy_score(y_test, bagging_predictions)
print("Bagging Accuracy:", bagging_accuracy)
'''
# Boosting
boosting_model = AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
boosting_model.fit(X_train, y_train)
boosting_predictions = boosting_model.predict(X_test)
boosting_accuracy = accuracy_score(y_test, boosting_predictions)
print("Boosting Accuracy:", boosting_accuracy)

'''# Stacking
estimators = [
    ('bagging', BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)),
    ('boosting', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42))
]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_model.fit(X_train, y_train)
stacking_predictions = stacking_model.predict(X_test)
stacking_accuracy = accuracy_score(y_test, stacking_predictions)
print("Stacking Accuracy:", stacking_accuracy)'''

Boosting Accuracy: 0.7608695652173914


c:\Users\hohih\ivh\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


'# Stacking\nestimators = [\n    (\'bagging\', BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)),\n    (\'boosting\', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42))\n]\nstacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())\nstacking_model.fit(X_train, y_train)\nstacking_predictions = stacking_model.predict(X_test)\nstacking_accuracy = accuracy_score(y_test, stacking_predictions)\nprint("Stacking Accuracy:", stacking_accuracy)'

In [23]:
# Make predictions on the test set
y_pred = boosting_predictions # rfe.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC:", roc_auc)

Accuracy: 0.7608695652173914
Precision: 0.75
Recall: 0.6428571428571429
F1-score: 0.6923076923076923
AUC-ROC: 0.7443257676902537


In [24]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[362  66]
 [110 198]]


In [25]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
num_classes=2
# Set the parameters for XGBoost
params = {
    'objective': 'multi:softmax',  # Set the objective function for multi-class classification
    'num_class': num_classes,  # Set the number of classes
    'eval_metric': 'merror'  # Set the evaluation metric
}

# Train the XGBoost model
num_rounds = 100  # Set the number of boosting rounds
xgb_model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test set
y_pred = xgb_model.predict(dtest)

# Convert the predicted labels to integers
y_pred = y_pred.astype(int)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5679347826086957


In [26]:
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)

# Train the SVM classifier
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6385869565217391


In [27]:
y_train = np.eye(num_classes)[y_train]
y_test = np.eye(num_classes)[y_test]
max_length = max([len(seq) for seq in X_train + X_test])
X_train = np.array([np.pad(seq, ((0, max_length - len(seq)), (0, 0)), 'constant') for seq in X_train])
X_test = np.array([np.pad(seq, ((0, max_length - len(seq)), (0, 0)), 'constant') for seq in X_test])

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (0,2)

In [34]:
from keras.layers import LSTM, Dense

In [35]:
model = Sequential()
model.add(LSTM(70, input_shape=(max_length, X_train.shape[2])))  # 50 is the number of LSTM units, adjust as needed
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

IndexError: tuple index out of range

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_labels)
print("Accuracy:", accuracy)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)
# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
(X_test[0].squeeze()), y_test[0]

## Neural Network training

In [ ]:
#from keras.models import Sequential
#from keras.layers import LSTM, Dense

In [36]:
# Define the feedforward neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to original labels
y_test_labels = np.argmax(y_test, axis=1)

# Compute the accuracy
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print("Accuracy:", accuracy)

# Compute the confusion matrix
cm = confusion_matrix(y_test_labels, y_pred_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

IndexError: tuple index out of range

In [ ]:
y_train.shape